In [89]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [90]:
df = pd.read_csv('workout_data.csv')

In [91]:
df.head(2)

,title,start_time,end_time,description,exercise_title,superset_id,exercise_notes,set_index,set_type,weight_kg,reps,distance_km,duration_seconds,rpe
0,Day 4: Arms,"10 Feb 2024, 19:57","10 Feb 2024, 21:34",NaN,Seated Incline Curl (Dumbbell),NaN,NaN,0,normal,12.0,9,NaN,NaN,NaN
1,Day 4: Arms,"10 Feb 2024, 19:57","10 Feb 2024, 21:34",NaN,Seated Incline Curl (Dumbbell),NaN,NaN,1,normal,12.0,6,NaN,NaN,NaN


In [92]:
muscle_group_dict = {
    'Seated Incline Curl (Dumbbell)': 'Arms',
    'Seated Palms Up Wrist Curl': 'Arms',
    'Triceps Extension (Dumbbell)': 'Arms',
    'Preacher Curl (Dumbbell)': 'Arms',
    'Triceps Pushdown': 'Arms',
    'Forearm Dumbell Curl': 'Arms',
    'Hammer Curl (Dumbbell)': 'Arms',
    'Knee Raise Parallel Bars': 'Abs',
    'Leg Press Horizontal (Machine)': 'Legs',
    'Pull Up': 'Back',
    'Seated Row (Machine)': 'Back',
    'Face Pull': 'Shoulders',
    'Leg Extension (Machine)': 'Legs',
    'Dumbell Skiers': 'Shoulders',
    'Shrug (Dumbbell)': 'Back',
    'Seated Cable Row - Bar Grip': 'Back',
    'Lat Pulldown (Machine)': 'Back',
    'Seated Calf Raise': 'Legs',
    'Incline Bench Press (Dumbbell)': 'Chest',
    'Shoulder Press (Dumbbell)': 'Shoulders',
    'Incline Bench Press (Barbell)': 'Chest',
    'Lateral Raise (Dumbbell)': 'Shoulders',
    'Cable Fly Crossovers': 'Chest',
    'Shoulder Press (Machine Plates)': 'Shoulders',
    'Torso Rotation': 'Abs'
}

df['muscle_group'] = df['exercise_title'].map(muscle_group_dict)



In [93]:
df.head(1)
df['start_time'].to_list()[1]
df['start_time'] = pd.to_datetime(df['start_time'], format='%d %b %Y, %H:%M')
df['end_time'] = pd.to_datetime(df['end_time'], format='%d %b %Y, %H:%M')

In [120]:
df.head()

,title,start_time,end_time,description,exercise_title,superset_id,exercise_notes,set_index,set_type,weight_kg,reps,distance_km,duration_seconds,rpe,muscle_group,start_hour,end_hour,duration
0,Day 4: Arms,2024-02-10 19:57:00,2024-02-10 21:34:00,NaN,Seated Incline Curl (Dumbbell),NaN,NaN,0,normal,12.0,9,NaN,NaN,NaN,Arms,19,21,97.0
1,Day 4: Arms,2024-02-10 19:57:00,2024-02-10 21:34:00,NaN,Seated Incline Curl (Dumbbell),NaN,NaN,1,normal,12.0,6,NaN,NaN,NaN,Arms,19,21,97.0
2,Day 4: Arms,2024-02-10 19:57:00,2024-02-10 21:34:00,NaN,Seated Incline Curl (Dumbbell),NaN,NaN,2,normal,12.0,6,NaN,NaN,NaN,Arms,19,21,97.0
3,Day 4: Arms,2024-02-10 19:57:00,2024-02-10 21:34:00,NaN,Seated Palms Up Wrist Curl,NaN,NaN,0,normal,8.0,14,NaN,NaN,NaN,Arms,19,21,97.0
4,Day 4: Arms,2024-02-10 19:57:00,2024-02-10 21:34:00,NaN,Seated Palms Up Wrist Curl,NaN,NaN,1,normal,8.0,14,NaN,NaN,NaN,Arms,19,21,97.0


In [131]:
# Do repartition plota

df_unique = df.drop_duplicates(subset='title')

minutes = [0] * 24

def calculate_minutes(row):
    global minutes
    start_time = row['start_time']
    end_time = row['end_time']
    for i in range(start_time.hour, end_time.hour + 1):
        if i == start_time.hour:
            minutes[i] += 60 - start_time.minute if start_time.minute != 0 else 0
        elif i == end_time.hour:
            minutes[i] += end_time.minute
        else:
            minutes[i] += 60

df_unique.apply(calculate_minutes, axis=1)
print(minutes)

total_minutes = sum(minutes)
percentage = [(x/total_minutes) * 100 for x in minutes]
fig = go.Figure(data=[go.Bar(
    x = [str(i) for i in range(24)],
    y = percentage
)])

fig.update_layout(title_text='Workout Time Distribution throughout the day',
                xaxis_title='Hour',
                yaxis_title='Percentage of time spent')

[0, 0, 0, 0, 0, 0, 0, 0, 0, 26, 60, 34, 0, 0, 0, 0, 0, 0, 0, 36, 117, 77, 57, 0]


In [147]:
# Do wykresu liniowego max ciężar (w zależności od powtórzeń w serii (albo bez tego)) w zależności od ćwiczenia

excercise = 'Incline Bench Press (Dumbbell)'

df_excercise = df[df['exercise_title'] == excercise]
df_grouped = df_excercise.groupby('title')[['weight_kg', 'start_time']].max().reset_index()
df_grouped['start_time'][0].date()


2024-02-03


In [163]:
df_weight = pd.read_csv('weight_data.csv')
df_weight.head()
df_weight_cleaned = df_weight.loc[df_weight['Measurement'] == 'Bodyweight']
df_weight_cleaned.head(3)


,Date,Time,Measurement,Value,Unit
0,2019-12-26,22:10:43,Bodyweight,60.0,kgs
1,2019-12-28,22:11:15,Bodyweight,62.6,kgs
2,2021-02-19,06:52:23,Bodyweight,64.9,kgs


In [170]:
fig = px.line(df_weight_cleaned, x='Date', y='Value')
fig

In [180]:
test = df_grouped['start_time'].dt.date
test

0    2024-02-03
1    2024-02-08
Name: start_time, dtype: object